<a href="https://colab.research.google.com/github/jongwoonalee/jongwoonalee.github.io/blob/main/Corner_Detection_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
코너 감지 알고리즘 (Corner Detection Algorithm)
==============================================

이 코드는 두 가지 주요 코너 감지 방법을 구현합니다:
1. 최소 고유값(Minimum Eigenvalue) 방법 - Shi-Tomasi 코너 감지기로도 알려짐
2. Harris 코너 측정 방법 - Harris와 Stephens에 의해 제안됨

코너 감지는 컴퓨터 비전에서 중요한 작업으로, 특징점 추출, 물체 인식, 이미지 정합 등에 활용됩니다.
"""

import numpy as np
import matplotlib.pyplot as plt
import time

def cornerDetect(img, winSize=7, type=0):
    """
    이미지에서 코너를 감지하는 함수입니다.

    매개변수:
        img: 그레이스케일 이미지 (2D 배열)
        winSize: 윈도우 크기 (기본값 7)
        type: 0=최소 고유값 방법, 1=Harris 방법 (기본값 0)

    반환값:
        corner_response: 코너 강도 맵 (입력 이미지와 같은 크기)
    """

    # 계산 정확도를 위해 이미지를 32비트 부동소수점으로 변환
    # 이유:
    # 1. 부동소수점은 소수점 이하 값을 처리할 수 있어 미분 계산에 더 적합
    # 2. 기울기 연산에서 음수 값을 다룰 수 있음
    # 3. 연산 과정에서 누적되는 오차를 줄일 수 있음
    # 4. 32비트는 64비트보다 메모리 효율적이면서 충분한 정밀도 제공
    img = img.astype(np.float32)

    # 단계 1: Sobel 커널 정의
    # Sobel 커널은 이미지의 기울기(gradient)를 계산하기 위한 필터
    # 기울기는 이미지의 픽셀 값이 얼마나 빠르게 변화하는지를 나타냄

    # a. 수평 기울기를 위한 Sobel 커널 (x방향 미분)
    # 이 커널은 수직 경계(세로선)를 강조합니다.
    # 열 방향으로 [-1, 0, 1] 패턴이 반복되어 수평 방향 변화 감지
    # 행 방향으로 [1, 2, 1] 가중치를 주어 노이즈 감소 효과
    sobel_x = np.array([[1, 0, -1],
                        [2, 0, -2],
                        [1, 0, -1]], dtype=np.float32)

    # b. 수직 기울기를 위한 Sobel 커널 (y방향 미분)
    # 이 커널은 수평 경계(가로선)를 강조합니다.
    # 행 방향으로 [1, 0, -1] 패턴이 반복되어 수직 방향 변화 감지
    # sobel_x를 90도 회전한 형태
    sobel_y = np.array([[1, 2, 1],
                        [0, 0, 0],
                        [-1, -2, -1]], dtype=np.float32)

    # 2D 컨볼루션 함수 정의 - 이미지에 필터 적용
    # 컨볼루션은 이미지 처리의 기본 연산으로, 필터(커널)를 이미지 위에 슬라이딩하며 적용
    def convolve2d(img, kernel):
        kh, kw = kernel.shape  # 커널의 높이와 너비

        # 패딩 계산: 컨볼루션 후에도 원본 이미지와 크기가 같도록 패딩
        # 커널 크기의 절반을 패딩으로 추가 (정수 나눗셈)
        pad_h, pad_w = kh // 2, kw // 2

        # 이미지 패딩 적용
        # 'reflect' 모드: 이미지의 경계를 거울처럼 반사하여 패딩
        # 이는 경계 주변에서 발생할 수 있는 인공물(artifacts)을 줄이는 효과
        padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')

        # 이미지 크기 추출
        h, w = img.shape

        # 출력 이미지 초기화 (입력과 같은 크기)
        out = np.zeros_like(img)

        # 이미지의 각 픽셀에 컨볼루션 적용
        # 이중 반복문을 통해 각 픽셀 위치에서:
        for i in range(h):
            for j in range(w):
                # 현재 픽셀을 중심으로 커널 크기의 영역(ROI) 추출
                # 슬라이딩 윈도우 방식으로 이미지 탐색
                region = padded[i:i+kh, j:j+kw]

                # 컨볼루션 계산: 영역과 커널의 원소별 곱의 합
                # 이는 행렬 곱셈이 아닌 원소별 곱셈(element-wise multiplication)과 합
                # 이 연산이 각 위치에서의 필터 응답 값을 계산
                out[i, j] = np.sum(region * kernel)

        return out

    # 단계 2: 이미지 기울기 계산
    # Sobel 필터를 이용해 x축과 y축 방향의 기울기 계산
    # 이 기울기는 이미지의 '변화율'을 나타내며, 경계나 코너 같은 특징을 검출하는 기초가 됨

    # 수평 방향(x축) 기울기 - 수직 경계를 강조
    Ix = convolve2d(img, sobel_x)

    # 수직 방향(y축) 기울기 - 수평 경계를 강조
    Iy = convolve2d(img, sobel_y)

    # 단계 3: 두 번째 모멘트 행렬(구조 텐서)의 구성요소 계산
    # 두 번째 모멘트 행렬 H는 다음과 같은 형태:
    # H = [[A, B],
    #      [B, C]]
    # 여기서:
    #   A = sum(Ix^2) - x방향 기울기의 제곱 합
    #   B = sum(Ix*Iy) - x와 y방향 기울기의 곱 합
    #   C = sum(Iy^2) - y방향 기울기의 제곱 합

    # 기울기의 제곱과 곱 계산
    Ix2 = Ix * Ix  # Ix^2: x방향 기울기의 제곱
    Iy2 = Iy * Iy  # Iy^2: y방향 기울기의 제곱
    Ixy = Ix * Iy  # Ix*Iy: x와 y방향 기울기의 곱

    # 박스 필터 함수 정의 - 윈도우 내에서 값을 평균화
    # 박스 필터는 모든 원소가 동일한 값을 가진 커널을 사용하는 단순한 형태의 필터
    # 이 필터는 지역적인 평활화(스무딩)를 수행, 노이즈 감소 효과
    def box_filter(img, winSize):
        # 정규화된 박스 필터 커널 생성
        # winSize × winSize 크기의 모든 원소가 1/(winSize*winSize)인 행렬
        # 이는 커널 영역 내 픽셀 값들의 평균을 계산
        k = np.ones((winSize, winSize), dtype=np.float32) / (winSize * winSize)

        # 박스 필터 적용 (컨볼루션 이용)
        return convolve2d(img, k)

    # 단계 4: 두 번째 모멘트 행렬의 각 구성요소에 박스 필터 적용
    # 윈도우 내에서 기울기 정보를 종합(스무딩)
    # 이 과정은 지역적 구조 정보를 파악하기 위함

    # A: x방향 기울기 제곱의 윈도우 내 평균
    A = box_filter(Ix2, winSize)

    # B: x와 y방향 기울기 곱의 윈도우 내 평균
    B = box_filter(Ixy, winSize)

    # C: y방향 기울기 제곱의 윈도우 내 평균
    C = box_filter(Iy2, winSize)

    # 단계 5: 코너 응답 함수 계산 (방법에 따라 다름)
    # 코너 감지 방법 선택: 최소 고유값 또는 Harris

    # 최소 고유값 방법 (Shi-Tomasi)
    if type == 0:
        # 행렬 H의 트레이스 계산 (대각선 요소의 합)
        trace = A + C

        # 행렬 H의 행렬식 계산 (AC - B^2)
        det = A * C - B * B

        # 고유값 계산을 위한 제곱근 항
        # 이차방정식 근의 공식 사용: λ = (trace ± √(trace^2 - 4*det))/2
        # 여기서 sqrt_term은 '판별식'에 해당
        sqrt_term = np.sqrt((A - C) ** 2 + 4 * B * B)

        # 최소 고유값 계산
        # 두 고유값 중 작은 값 λ_min = (trace - sqrt_term)/2
        lambda_min = 0.5 * (trace - sqrt_term)

        # 최소 고유값을 코너 응답으로 사용
        # 두 고유값이 모두 큰 경우(즉, 최소 고유값도 큰 경우) 코너로 판단
        corner_response = lambda_min

    # Harris 코너 방법
    elif type == 1:
        # Harris 민감도 파라미터 k
        # 0.04~0.06 범위가 일반적, 경험적으로 0.05가 좋은 결과를 제공
        k = 0.05

        # 행렬 H의 행렬식 계산 (AC - B^2)
        det = A * C - B * B

        # 행렬 H의 트레이스 계산 (대각선 요소의 합)
        trace = A + C

        # Harris 응답 함수: R = det(H) - k*(trace(H))^2
        # 이 식은 두 고유값의 곱에서 그 합의 제곱에 k를 곱한 값을 뺀 것
        # R이 큰 양수 -> 코너, R이 큰 음수 -> 경계, |R|이 작음 -> 평평한 영역
        corner_response = det - k * (trace ** 2)

    # 잘못된 매개변수 type에 대한 예외 처리
    else:
        raise ValueError("Invalid type: 0=Eigenvalue, 1=Harris")

    # 계산된 코너 응답 맵 반환
    # 입력 이미지와 같은 크기의 2D 배열로, 각 픽셀의 '코너 강도'를 나타냄
    return corner_response

# 이미지 윈도우 크기에 따른 성능 비교 실험 코드
def compare_corner_detection(img):
    """
    다양한 윈도우 크기에서 두 코너 감지 방법의 성능과 실행 시간을 비교합니다.

    매개변수:
        img: 그레이스케일 입력 이미지

    출력:
        1. 다양한 윈도우 크기에 대한 두 방법의 코너 응답 맵 시각화
        2. 두 방법의 실행 시간 비교 그래프
    """

    # 테스트할 윈도우 크기 목록
    # 작은 크기부터 큰 크기까지 다양한 윈도우 크기 테스트
    window_sizes = [3, 5, 7, 9, 11, 15]

    # 실행 시간 및 결과 저장을 위한 리스트
    timings_eigen = []      # 최소 고유값 방법의 실행 시간
    timings_harris = []     # Harris 방법의 실행 시간
    corner_maps_eigen = []  # 최소 고유값 방법의 코너 응답 맵
    corner_maps_harris = [] # Harris 방법의 코너 응답 맵

    # 코너 응답 맵 정규화 함수
    # 시각화를 위해 모든 코너 응답 값을 [0, 1] 범위로 변환
    def normalize(arr):
        # 최소값을 0으로 이동 (모든 값에서 최소값을 뺌)
        arr = arr - np.min(arr)

        # 최대값을 1로 조정 (모든 값을 최대값으로 나눔)
        arr = arr / np.max(arr)

        # [0, 1] 범위의 배열 반환
        return arr

    # 각 윈도우 크기에 대해 두 방법으로 코너 감지 실행 및 시간 측정
    for win in window_sizes:
        # 최소 고유값 방법 (type=0)
        start = time.time()  # 시작 시간 기록
        result_eigen = cornerDetect(img, winSize=win, type=0)  # 코너 감지 실행
        timings_eigen.append(time.time() - start)  # 실행 시간 저장
        corner_maps_eigen.append(result_eigen)  # 코너 응답 맵 저장

        # Harris 방법 (type=1)
        start = time.time()  # 시작 시간 기록
        result_harris = cornerDetect(img, winSize=win, type=1)  # 코너 감지 실행
        timings_harris.append(time.time() - start)  # 실행 시간 저장
        corner_maps_harris.append(result_harris)  # 코너 응답 맵 저장

    # 코너 응답 맵 시각화
    # 각 윈도우 크기와 방법에 대한 코너 응답 맵을 그리드로 표시
    plt.figure(figsize=(16, 8))

    # 모든 윈도우 크기에 대해 반복
    for i, win in enumerate(window_sizes):
        # 최소 고유값 방법 결과 표시 (상단 행)
        plt.subplot(2, len(window_sizes), i + 1)
        plt.imshow(normalize(corner_maps_eigen[i]), cmap='hot')  # 'hot' 컬러맵: 빨간색=강한 코너
        plt.title(f"Eigen\nwin={win}")
        plt.axis('off')  # 축 표시 제거

        # Harris 방법 결과 표시 (하단 행)
        plt.subplot(2, len(window_sizes), len(window_sizes) + i + 1)
        plt.imshow(normalize(corner_maps_harris[i]), cmap='hot')
        plt.title(f"Harris\nwin={win}")
        plt.axis('off')

    # 그래프 레이아웃 조정 및 제목 추가
    plt.tight_layout()
    plt.suptitle("Corner Maps (Eigenvalue, Harris)", fontsize=16, y=1.03)
    plt.show()

    # 실행 시간 비교 그래프 생성
    plt.figure(figsize=(8, 5))

    # 최소 고유값 방법의 실행 시간 그래프 (원형 마커)
    plt.plot(window_sizes, timings_eigen, marker='o', label='Minimum Eigenvalue')

    # Harris 방법의 실행 시간 그래프 (사각형 마커)
    plt.plot(window_sizes, timings_harris, marker='s', label='Harris Corner')

    # 그래프 라벨 및 제목 설정
    plt.xlabel("Window Size")
    plt.ylabel("Execution Time (seconds)")
    plt.title("Corner Detection Timing Comparison")
    plt.legend()
    plt.grid(True)
    plt.show()

    # 결과 분석 출력
    print("실험 결과 분석:")
    print("\n1. 윈도우 크기와 코너 감지 성능:")
    print("  - 작은 윈도우(3, 5): 많은 작은 코너를 감지하지만 노이즈와 분절된 응답이 많음")
    print("  - 중간 윈도우(7, 9): 노이즈를 줄이면서 모든 의미 있는 코너를 명확하게 감지하는 좋은 균형")
    print("  - 큰 윈도우(11, 15): 응답이 더 부드럽지만 미세한 코너를 놓치거나 조밀한 패턴의 코너 세부사항이 흐려짐")

    # Harris가 더 빠른 퍼센트 계산
    percent_faster = [(t_eigen - t_harris) / t_eigen * 100 for t_eigen, t_harris in zip(timings_eigen, timings_harris)]
    avg_percent = sum(percent_faster) / len(percent_faster)

    print("\n2. 실행 시간 비교:")
    print(f"  - Harris 방법이 최소 고유값 방법보다 평균 {avg_percent:.1f}% 더 빠름")
    print("  - 최소 고유값 방법이 더 느린 이유는 제곱근 계산이 필요하기 때문(계산 비용이 더 높음)")
    print("  - 두 방법 모두 동일한 전처리 단계를 공유하므로 기본 성능은 비슷함")

# 사용 예시
"""
import cv2

# 이미지 불러오기
img = cv2.imread('image.jpg', cv2.IMREAD_GRAYSCALE)

# 코너 감지 성능 비교 실행
compare_corner_detection(img)
"""

# 코너 감지의 이론적 배경
"""
코너 감지의 이론적 배경:
=====================

1. 코너의 정의:
   코너는 이미지에서 두 방향 이상으로 큰 강도 변화가 있는 지점입니다.
   예: 체스판의 꼭지점, 물체의 모서리 등

2. 구조 텐서(Structure Tensor)의 의미:
   구조 텐서 H는 지역 내 픽셀 값의 변화 방향과 크기를 설명합니다.
   - H의 고유값(λ₁, λ₂)은 변화의 크기를 나타냅니다.
   - H의 고유벡터는 변화의 방향을 나타냅니다.

3. 고유값 해석:
   - λ₁, λ₂ 모두 작음 → 평평한 영역(변화 없음)
   - λ₁ >> λ₂, λ₂ 작음 → 경계(한 방향으로만 변화)
   - λ₁, λ₂ 모두 큼 → 코너(여러 방향으로 변화)

4. 윈도우 크기의 영향:
   - 작은 윈도우: 세부적인 코너 감지, 노이즈에 민감
   - 큰 윈도우: 더 안정적인 결과, 미세한 코너를 놓칠 수 있음
   - 최적의 윈도우 크기는 이미지 내용과 적용에 따라 달라짐

5. 두 방법 비교:
   - 최소 고유값(Shi-Tomasi): 직관적, 이론적으로 견고, 계산은 더 복잡
   - Harris: 계산 효율적, 민감도 조절 가능, 널리 사용됨
"""